# Import Modules

In [1]:
import os

In [2]:
!pip install transformers

     |████████████████████████████████| 1.4MB 8.4MB/s 
     |████████████████████████████████| 890kB 31.6MB/s 
     |████████████████████████████████| 2.9MB 42.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=6d3e2308f919f107f4bba0447496fb7c8eabc571ff8aef9c845a0c7d6212801a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
from transformers import BertTokenizer, XLNetTokenizer, RobertaTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [4]:
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, f1_score, accuracy_score

In [5]:
import tensorflow as tf
# Get the GPU device name.
device_name = tf.test.gpu_device_name()
# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [6]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [7]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tqdm import tqdm , trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt

In [8]:
# Check to confirm the specific GPU model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)


'Tesla P100-PCIE-16GB'

In [9]:
from transformers import *
import os

In [10]:
# error label CO -> correct sentence 
ERR_LBLS = ["CO", "GM", "LX", "OT"]


# Get Model from Drive

### Old Model

In [ ]:
!cp  '/content/gdrive/My Drive/NLP/bert-based-uncased-EDC-final.pth' .

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = len(ERR_LBLS), output_hidden_states=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Restore saved model

In [ ]:
model.load_state_dict(torch.load('bert-based-uncased-EDC-final.pth'))

<All keys matched successfully>

In [ ]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

## Get Model from Gitlab

In [14]:
!git clone https://gitlab.com/vibss2397/nlp.git

Cloning into 'nlp'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 35 (delta 12), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (35/35), done.
Checking out files: 100% (7/7), done.


In [16]:
import torch.nn as nn

In [17]:
class CustomBERTModel(nn.Module):
    def __init__(self, n_heads = 4):
        super(CustomBERTModel, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        for name, param in self.bert.named_parameters():
            if(name.find('11')>=0 or name.find('pooler')>=0):
                param.requires_grad = True
            else:
                param.requires_grad = True
          # self.classifier = nn.Linear()
          # add your additional layers here, for example a dropout layer followed by a linear classification head
        self.dropout = nn.Dropout(0.5)
        self.out1 = nn.Linear(768, 1)
        self.out2 = nn.Linear(768, 1)
        self.out3 = nn.Linear(768, 1)
        self.out4 = nn.Linear(768, 1)
    def forward(self, input_ids, token_type_ids, attention_mask):
        pooled_output = self.bert(
               input_ids, 
               attention_mask,
               token_type_ids
          )
          # out1 = self.dropout(pooled_output)
        out1 = self.out1(pooled_output[1])
          
          # out2 = self.dropout(pooled_output)
        out2 = self.out2(pooled_output[1])
          
          # out3 = self.dropout(pooled_output)
        out3 = self.out3(pooled_output[1])

          # out4 = self.dropout(pooled_output)
        out4 = self.out4(pooled_output[1])
          # dropout = self.dropout(pooled_output)
          # out1 = self.out1(dropout)
          # out1 = self.relu(out1)
          
          # logits = self.out2(dropout)
        return out1, out2, out3, out4

In [18]:
!ls nlp

fce_final.csv  final		  ref_diff.gu-en.en
fce_old.csv    hyp_diff.gu-en.en  Untitled.ipynb


In [20]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CustomBERTModel().to(device)
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

model.load_state_dict(torch.load('nlp/final/entire-less_decay.pth'))

<All keys matched successfully>

# Start Testing (change path as required)

In [21]:
primaryPath = "/content/gdrive/My Drive/NLP/Translations/"

In [22]:
MAX_LEN = 512
BATCH_SIZE = 5

## XLM Generation

### Data File Maps

#### Layer Map

In [108]:
dataFolder = "xlm/final/layers/layer5"
dataPath = primaryPath + dataFolder

ref_file_name="ref.gu-en.test.txt"

In [109]:
general_file_name_format = ".gu-en.test.txt"
# layer 0
# budget_file_map = {
#     "1000": "hyp8",
#     "2000": "hyp0",
#     "5000": "hyp0",
#     "10000": "hyp0",
#     "20000": "hyp7",
#     "50000": "hyp1",
#     "70000": "hyp3",
# }

# layer 1
# budget_file_map = {
#     "1000": "hyp4",
#     "2000": "hyp7",
#     "5000": "hyp3",
#     "10000": "hyp0",
#     "20000": "hyp0",
#     "50000": "hyp0",
#     "70000": "hyp0",
# }
# layer 2
# budget_file_map = {
#     "1000": "hyp8",
#     "2000": "hyp2",
#     "5000": "hyp0",
#     "10000": "hyp0",
#     "20000": "hyp0",
#     "50000": "hyp3",
#     "70000": "hyp0",
# }

# layer 3
# budget_file_map = {
#     "1000": "hyp9",
#     "2000": "hyp2",
#     "5000": "hyp0",
#     "10000": "hyp2",
#     "20000": "hyp3",
#     "50000": "hyp1",
#     "70000": "hyp3",
# }

# layer 4
# budget_file_map = {
#     "1000": "hyp8",
#     "2000": "hyp3",
#     "5000": "hyp0",
#     "10000": "hyp0",
#     "20000": "hyp0",
#     "50000": "hyp0",
#     "70000": "hyp0",
# }

# layer 5
# budget_file_map = {
#     "1000": "hyp9",
#     "2000": "hyp6",
#     "5000": "hyp0",
#     "10000": "hyp0",
#     "20000": "hyp3",
#     "50000": "hyp0",
#     "70000": "hyp0",
# }

#### Head Map

In [133]:
dataFolder = "xlm/final/heads/head7"
dataPath = primaryPath + dataFolder

In [134]:
general_file_name_format = ".gu-en.test.txt"
# head_0
# budget_file_map = {
#     "1000": "hyp9",
#     "2000": "hyp3",
#     "5000": "hyp3",
#     "10000": "hyp0",
#     "20000": "hyp6",
#     "50000": "hyp1",
#     "70000": "hyp0",
# }

# head 1
# budget_file_map = {
#     "1000": "hyp8",
#     "2000": "hyp5",
#     "5000": "hyp1",
#     "10000": "hyp0",
#     "20000": "hyp1",
#     "50000": "hyp0",
#     "70000": "hyp2",
# }
# head 2
# budget_file_map = {
#     "1000": "hyp9",
#     "2000": "hyp2",
#     "5000": "hyp5",
#     "10000": "hyp0",
#     "20000": "hyp0",
#     "50000": "hyp0",
#     "70000": "hyp0",
# }

# head 3
# budget_file_map = {
#     "1000": "hyp4",
#     "2000": "hyp6",
#     "5000": "hyp8",
#     "10000": "hyp3",
#     "20000": "hyp0",
#     "50000": "hyp0",
#     "70000": "hyp1",
# }

# head 4
# budget_file_map = {
#     "1000": "hyp8",
#     "2000": "hyp6",
#     "5000": "hyp0",
#     "10000": "hyp1",
#     "20000": "hyp4",
#     "50000": "hyp3",
# }

# head 5
# budget_file_map = {
#     "1000": "hyp9",
#     "2000": "hyp5",
#     "5000": "hyp1",
#     "10000": "hyp0",
#     "20000": "hyp0",
#     "50000": "hyp0",
#     "70000": "hyp0",
# }

# head 6
# budget_file_map = {
#     "1000": "hyp8",
#     "2000": "hyp4",
#     "5000": "hyp1",
#     "10000": "hyp2",
#     "20000": "hyp0",
#     "50000": "hyp0",
#     "70000": "hyp1",
# }

# head 7
# budget_file_map = {
#     "1000": "hyp8",
#     "2000": "hyp3",
#     "5000": "hyp3",
#     "10000": "hyp0",
#     "20000": "hyp2",
#     "50000": "hyp0",
#     "70000": "hyp1",
# }

#### Best Map

In [137]:
dataFolder = "xlm/final/random/"
dataPath = primaryPath + dataFolder

In [138]:
general_file_name_format = ".gu-en.test.txt"

budget_file_map = {
    "1000": "hyp7",
    "2000": "hyp6",
    "5000": "hyp0",
    "10000": "hyp1",
    "20000": "hyp0",
    "50000": "hyp0",
    "70000": "hyp0",
}


### Testing

#### Sample test

In [ ]:
translations = os.listdir(dataPath)

for i in tqdm(range(len(translations))):
  translation = translations[i]
  print("trans", translation)
  if(translation.startswith(".")):
    continue
  subdir = os.listdir(dataPath+"/"+translation)
  
  sub = subdir[0]
  if(subdir[0].startswith(".")):
    sub = subdir[1]
  print(sub, type(sub)) # budget
  childrenPath = dataPath+"/"+translation + "/" + sub + "/hypotheses/"
  children = os.listdir(childrenPath)
  print(children)
  filename = childrenPath + budget_file_map[sub]+general_file_name_format
  print("imp file: ", filename)
  break

  0%|          | 0/4 [00:00<?, ?it/s]

trans unsupMT_guen_head0_budget5000_epoch_size50000
5000 <class 'str'>


  0%|          | 0/4 [00:01<?, ?it/s]

['hyp0.gu-en.valid.txt', 'ref.en-gu.valid.txt', 'hyp0.gu-en.test.txt', 'hyp0.en-gu.test.txt', 'hyp1.gu-en.test.txt', 'hyp1.en-gu.test.txt', 'ref.en-gu.test.txt', 'hyp1.en-gu.valid.txt', 'hyp1.gu-en.valid.txt', 'ref.gu-en.test.txt', 'hyp0.en-gu.valid.txt', 'ref.gu-en.valid.txt']
imp file:  /content/gdrive/My Drive/NLP/Translations/xlm/Dec1/heads_acc/head0/unsupMT_guen_head0_budget5000_epoch_size50000/5000/hypotheses/hyp1.gu-en.test.txt


#### Actual Label Generation

In [113]:
def encode_sentence(sentence, max_len):
    return bert_tokenizer.encode_plus(
                        sentence,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = max_len,           # Pad & truncate all sentences.
                        padding = 'max_length',
                        truncation = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt'    # Return pytorch tensors.
                   )

In [135]:


translations = os.listdir(dataPath)

for i in tqdm(range(len(translations))):
  translation = translations[i]
  print("trans", translation)
  if(translation.startswith(".")):
    continue
  subdir = os.listdir(dataPath+"/"+translation)
  print(subdir)
  sub = subdir[0]
  if(subdir[0].startswith(".")):
    sub = subdir[1]
  childrenPath = dataPath+"/"+translation + "/" + sub + "/hypotheses/"
  children = os.listdir(childrenPath)

  if not sub in budget_file_map:
    continue
  
  filename = childrenPath + budget_file_map[sub]+general_file_name_format
  print("file: ", filename)
  # f = open(filename)
  # content = f.read().split("\n")
  
  # a = content
  with open(filename) as f:
    a = f.readlines()
  a = [ele.split('\n')[0] for ele in a]
  model.eval()
  predictions = []

  counter = 0
  # print("No. of Sentences = ",len(a))
  for i in range(len(a)):
      sentence = a[i]
      encoded = encode_sentence(sentence, 353)
      for ele in encoded:
          encoded[ele] = encoded[ele].to(device)
      with torch.no_grad():
          prediction = model(**encoded)
      final = np.array(torch.sigmoid(torch.cat(prediction).reshape(4)).data.cpu().numpy() > 0.5, dtype=np.int)
      predictions.append([a[i], final])
      counter+=1  

  output = []
  for i in range(len(predictions)):
    o = {"sentence": predictions[i][0]}
    pred_bools = predictions[i][1]
    # print(pred_bools)
    for j in range(len(pred_bools)):
      o[ERR_LBLS[j]] = pred_bools[j]
    # print(o)

    output.append(o)
  
  output_df = pd.DataFrame(output)
  # print(output_df)
  # print(output_df.sum(axis = 0, skipna = False))
  res_filepath = childrenPath + "/"
  res_filename = res_filepath + "error_predictions.csv"
  output_df.to_csv(res_filename)
  

























  0%|          | 0/7 [00:00<?, ?it/s]

trans unsupMT_guen_head7_budget2000_epoch_size10000
['2000']
file:  /content/gdrive/My Drive/NLP/Translations/xlm/final/heads/head7/unsupMT_guen_head7_budget2000_epoch_size10000/2000/hypotheses/hyp3.gu-en.test.txt


























 14%|█▍        | 1/7 [00:36<03:41, 36.99s/it]

trans unsupMT_guen_head7_budget1000_epoch_size10000
['1000']
file:  /content/gdrive/My Drive/NLP/Translations/xlm/final/heads/head7/unsupMT_guen_head7_budget1000_epoch_size10000/1000/hypotheses/hyp8.gu-en.test.txt


























 29%|██▊       | 2/7 [01:13<03:04, 36.97s/it]

...

























 71%|███████▏  | 5/7 [03:04<01:13, 36.97s/it]

trans unsupMT_guen_head7_budget10000_epoch_size100000
['10000']
file:  /content/gdrive/My Drive/NLP/Translations/xlm/final/heads/head7/unsupMT_guen_head7_budget10000_epoch_size100000/10000/hypotheses/hyp0.gu-en.test.txt


























 86%|████████▌ | 6/7 [03:41<00:36, 36.98s/it]

trans unsupMT_guen_head7_budget50000_epoch_size100000
['50000']
file:  /content/gdrive/My Drive/NLP/Translations/xlm/final/heads/head7/unsupMT_guen_head7_budget50000_epoch_size100000/50000/hypotheses/hyp0.gu-en.test.txt


























100%|██████████| 7/7 [04:18<00:00, 36.99s/it]


### For Random Layer

In [147]:
translations = os.listdir(dataPath)
for i in tqdm(range(len(translations))):
  translation = translations[i]
  print("trans", translation)
  if(translation.startswith(".")):
    continue
  
  subdir = os.listdir(dataPath+"/"+translation)
  print(subdir)
  sub = subdir[0]
  if(subdir[0].startswith(".")):
    sub = subdir[1]
  
  childrenPath = dataPath+"/"+translation + "/" + sub + "/hypotheses/"
  print(childrenPath)

  if not sub in budget_file_map:
    continue
  
  filename = childrenPath + budget_file_map[sub]+general_file_name_format
  print("file: ", filename)
  
  
  with open(filename) as f:
    a = f.readlines()
  a = [ele.split('\n')[0] for ele in a]
  model.eval()
  predictions = []

  counter = 0
  # print("No. of Sentences = ",len(a))
  for i in range(len(a)):
      sentence = a[i]
      encoded = encode_sentence(sentence, 353)
      for ele in encoded:
          encoded[ele] = encoded[ele].to(device)
      with torch.no_grad():
          prediction = model(**encoded)
      final = np.array(torch.sigmoid(torch.cat(prediction).reshape(4)).data.cpu().numpy() > 0.5, dtype=np.int)
      predictions.append([a[i], final])
      counter+=1  

  output = []
  for i in range(len(predictions)):
    o = {"sentence": predictions[i][0]}
    pred_bools = predictions[i][1]
    # print(pred_bools)
    for j in range(len(pred_bools)):
      o[ERR_LBLS[j]] = pred_bools[j]
    # print(o)

    output.append(o)
  
  output_df = pd.DataFrame(output)
  # print(output_df)
  # print(output_df.sum(axis = 0, skipna = False))
  res_filepath = childrenPath + "/"
  res_filename = res_filepath + "error_predictions.csv"
  output_df.to_csv(res_filename)
  


























  0%|          | 0/7 [00:00<?, ?it/s]

trans unsupMT_guen_budget1000_epoch_size10000
['1000']
/content/gdrive/My Drive/NLP/Translations/xlm/final/random//unsupMT_guen_budget1000_epoch_size10000/1000/hypotheses/
file:  /content/gdrive/My Drive/NLP/Translations/xlm/final/random//unsupMT_guen_budget1000_epoch_size10000/1000/hypotheses/hyp7.gu-en.test.txt



























 14%|█▍        | 1/7 [00:36<03:40, 36.71s/it]

trans unsupMT_guen_budget10000_epoch_size100000
['10000']
/content/gdrive/My Drive/NLP/Translations/xlm/final/random//unsupMT_guen_budget10000_epoch_size100000/10000/hypotheses/
file:  /content/gdrive/My Drive/NLP/Translations/xlm/final/random//unsupMT_guen_budget10000_epoch_size100000/10000/hypotheses/hyp1.gu-en.test.txt



























 29%|██▊       | 2/7 [01:14<03:04, 36.97s/it]

...


























 71%|███████▏  | 5/7 [03:06<01:14, 37.24s/it]

trans unsupMT_guen_budget20000_epoch_size100000
['20000']
/content/gdrive/My Drive/NLP/Translations/xlm/final/random//unsupMT_guen_budget20000_epoch_size100000/20000/hypotheses/
file:  /content/gdrive/My Drive/NLP/Translations/xlm/final/random//unsupMT_guen_budget20000_epoch_size100000/20000/hypotheses/hyp0.gu-en.test.txt



























 86%|████████▌ | 6/7 [03:43<00:37, 37.27s/it]

trans unsupMT_guen_budget5000_epoch_size50000
['5000']
/content/gdrive/My Drive/NLP/Translations/xlm/final/random//unsupMT_guen_budget5000_epoch_size50000/5000/hypotheses/
file:  /content/gdrive/My Drive/NLP/Translations/xlm/final/random//unsupMT_guen_budget5000_epoch_size50000/5000/hypotheses/hyp0.gu-en.test.txt



























100%|██████████| 7/7 [04:21<00:00, 37.34s/it]


#### Old

In [41]:

translations = os.listdir(dataPath)

for i in tqdm(range(len(translations))):
  translation = translations[i]
  print("trans", translation)
  if(translation.startswith(".")):
    continue
  subdir = os.listdir(dataPath+"/"+translation)
  print(subdir)
  sub = subdir[0]
  if(subdir[0].startswith(".")):
    sub = subdir[1]
  childrenPath = dataPath+"/"+translation + "/" + sub + "/hypotheses/"
  children = os.listdir(childrenPath)

  if not sub in budget_file_map:
    continue
  
  filename = childrenPath + budget_file_map[sub]+general_file_name_format
  print("file: ", filename)
  f = open(filename)
  content = f.read().split("\n")
  
  actual_test_sentences = content

  actual_test_encodings = tokenizer.batch_encode_plus(actual_test_sentences,max_length=MAX_LEN, padding="max_length", truncation=True)
  actual_test_input_ids = actual_test_encodings['input_ids']
  actual_test_token_type_ids = actual_test_encodings['token_type_ids']
  actual_test_attention_masks = actual_test_encodings['attention_mask']

  # Make tensors out of data
  actual_test_inputs = torch.tensor(actual_test_input_ids)
  actual_test_masks = torch.tensor(actual_test_attention_masks)
  actual_test_token_types = torch.tensor(actual_test_token_type_ids)
  # Create test dataloader
  actual_test_data = TensorDataset(actual_test_inputs, actual_test_masks, actual_test_token_types)
  actual_test_sampler = SequentialSampler(actual_test_data)
  actual_test_dataloader = DataLoader(actual_test_data, sampler=actual_test_sampler, batch_size=BATCH_SIZE)

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  #track variables
  logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]

  # Predict
  for i, batch in enumerate(actual_test_dataloader):
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_token_types = batch
    with torch.no_grad():
      # Forward pass
      outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      b_logit_pred = outs[0]
      pred_label = torch.sigmoid(b_logit_pred)

      b_logit_pred = b_logit_pred.detach().cpu().numpy()
      pred_label = pred_label.to('cpu').numpy()
      
    tokenized_texts.append(b_input_ids)
    logit_preds.append(b_logit_pred)
    pred_labels.append(pred_label)

  # Flatten outputs
  pred_labels = [item for sublist in pred_labels for item in sublist]
  pred_bools = [pl>0.50 for pl in pred_labels]

  output = []
  for i in range(len(actual_test_sentences)):
    o = {"sentence": actual_test_sentences[i]}
    for j in range(len(pred_bools[i])):
      o[ERR_LBLS[j]] = pred_bools[i][j]
    output.append(o)
  
  output_df = pd.DataFrame(output)
  res_filepath = childrenPath + "/"
  res_filename = res_filepath + "error_predictions.csv"
  output_df.to_csv(res_filename)
      
      
  

  0%|          | 0/7 [00:00<?, ?it/s]

trans unsupMT_guen_layer5_budget20000_epoch_size100000
['20000']
file:  /content/gdrive/My Drive/NLP/Translations/xlm/final/layers/layer5/unsupMT_guen_layer5_budget20000_epoch_size100000/20000/hypotheses/hyp3.gu-en.test.txt


 14%|█▍        | 1/7 [00:45<04:34, 45.67s/it]

trans unsupMT_guen_layer5_budget70000_epoch_size100000
['70000']
file:  /content/gdrive/My Drive/NLP/Translations/xlm/final/layers/layer5/unsupMT_guen_layer5_budget70000_epoch_size100000/70000/hypotheses/hyp0.gu-en.test.txt


 29%|██▊       | 2/7 [01:30<03:47, 45.52s/it]

...

 71%|███████▏  | 5/7 [03:46<01:30, 45.40s/it]

trans unsupMT_guen_layer5_budget2000_epoch_size10000
['2000']
file:  /content/gdrive/My Drive/NLP/Translations/xlm/final/layers/layer5/unsupMT_guen_layer5_budget2000_epoch_size10000/2000/hypotheses/hyp6.gu-en.test.txt


 86%|████████▌ | 6/7 [04:32<00:45, 45.46s/it]

trans unsupMT_guen_layer5_budget1000_epoch_size10000
['1000']
file:  /content/gdrive/My Drive/NLP/Translations/xlm/final/layers/layer5/unsupMT_guen_layer5_budget1000_epoch_size10000/1000/hypotheses/hyp9.gu-en.test.txt


100%|██████████| 7/7 [05:18<00:00, 45.47s/it]
